In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
import pickle
import json
from imblearn.over_sampling import SMOTE
import plotly.express as px
import seaborn as sns


# I - Préparation des Données 

In [29]:
# Extraction des données d’intérêt
file_arbre = 'Data_Arbre.csv'
arbre = pd.read_csv(file_arbre)
print("Dimensions des données de 1 : ", arbre.shape)

# Les colonnes pertinentes
colonnes_pertinentes = ["longitude", "latitude", "haut_tot", "haut_tronc", "tronc_diam", "fk_arb_etat", "age_estim"]
data_arbre = arbre[colonnes_pertinentes].copy()
# Vérification des dimensions des données de test
print("Dimensions des données de 2 : ", data_arbre.shape)

# Filtrage pour inclure uniquement 'Essouché' et 'Non essouché'
data_arbre = data_arbre[data_arbre['fk_arb_etat'].isin(['Essouché', 'Non essouché'])]

# Vérification du nombre de chaque catégorie
#print("Nombre de 'Essouché' : ", (data_arbre['fk_arb_etat'] == 'Essouché').sum())
#print("Nombre de 'Non essouché' : ", (data_arbre['fk_arb_etat'] == 'Non essouché').sum())

# Vérification des valeurs manquantes dans la colonne 'fk_arb_etat'
#print("Nombre de valeurs manquantes dans 'fk_arb_etat' : ", data_arbre['fk_arb_etat'].isna().sum())

# Transformation de 'fk_arb_etat' en variable binaire
data_arbre['fk_arb_etat'] = (data_arbre['fk_arb_etat'] == 'Essouché').astype(int)

# Sauvegarde des données filtrées
with open('Encodage', 'wb') as f:
    pickle.dump(data_arbre, f)

print("Dimensions des données de 3 : ", data_arbre.shape)

# Séparation des caractéristiques et de la cible
x = data_arbre.drop(columns=['fk_arb_etat'])
y = data_arbre['fk_arb_etat']

print("Dimensions des données de 4 : ", data_arbre.shape)
# Séparation en ensembles d'entraînement (60%), de validation (25%) et de test (15%)
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)

# Colonnes des coordonnées
coordonnes = ['longitude', 'latitude']

# Séparer les colonnes des coordonnées pour ne pas les normaliser
x_train_coords = x_train[coordonnes]
x_val_coords = x_val[coordonnes]
x_test_coords = x_test[coordonnes]
x_train_features = x_train.drop(columns=coordonnes)
x_val_features = x_val.drop(columns=coordonnes)
x_test_features = x_test.drop(columns=coordonnes)

# Mise à l'échelle des caractéristiques
scaler = StandardScaler()
x_train_scaled_fit = scaler.fit_transform(x_train_features)
x_val_scaled_fit = scaler.transform(x_val_features)
x_test_scaled_fit = scaler.transform(x_test_features)

# Sauvegarde du scaler pour la normalisation
with open('Normalisation', 'wb') as f:
    pickle.dump(scaler, f)

# Convertir les résultats en DataFrame
x_train_scaled_features = pd.DataFrame(x_train_scaled_fit, columns=x_train_features.columns, index=x_train.index)
x_val_scaled_features = pd.DataFrame(x_val_scaled_fit, columns=x_val_features.columns, index=x_val.index)
x_test_scaled_features = pd.DataFrame(x_test_scaled_fit, columns=x_test_features.columns, index=x_test.index)

# Concaténer les données normalisées avec les coordonnées
x_train_scaled = pd.concat([x_train_coords, x_train_scaled_features], axis=1)
x_val_scaled = pd.concat([x_val_coords, x_val_scaled_features], axis=1)
x_test_scaled = pd.concat([x_test_coords, x_test_scaled_features], axis=1)

# Sauvegarde des données de test au format JSON
test_data = pd.concat([x_test_scaled.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_data.to_json('test_data.json', orient='records')

# Vérification des dimensions des données de test
#print("Dimensions des données de test : ", test_data.shape)

# Affichage des données de test et vérification des valeurs manquantes
#print("Valeurs manquantes dans les données de test : ")
#print(test_data.isna().sum())
print(test_data)


Dimensions des données de 1 :  (7409, 20)
Dimensions des données de 2 :  (7409, 7)
Dimensions des données de 3 :  (192, 7)
Dimensions des données de 4 :  (192, 7)
    longitude   latitude  haut_tot  haut_tronc  tronc_diam  age_estim  \
0    3.290751  49.848312  1.147136    0.175604    3.420865   7.121579   
1    3.314179  49.843834 -0.971321   -0.617867   -1.479382  -1.185090   
2    3.280806  49.865129 -0.838918   -1.411339   -1.401601  -1.185090   
3    3.286612  49.844846  0.087907    2.159284   -0.794903  -0.511576   
4    3.297266  49.862782  0.749925   -0.221131    0.387378  -0.062567   
5    3.297946  49.864024 -0.309303   -1.411339   -0.903798  -0.511576   
6    3.294984  49.850041  1.014732    0.175604    1.320759   1.733470   
7    3.292854  49.851386  1.544346    2.556020    2.409703   3.978515   
8    3.293294  49.851225  2.073960    3.746228    1.242977   1.733470   
9    3.294641  49.850508  1.014732    3.746228    0.931850   1.733470   
10   3.310065  49.855100 -0.176900

# II - Apprentissage Supervisé pour la régression classification

## Stochastic Gradient Descent

In [21]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train_scaled, y_train)

# Prédiction
y_pred = sgd_clf.predict(x_val_scaled)

# Calcul de la matrice de confusion sur l'ensemble de validation
matrix = confusion_matrix(y_val, y_pred, normalize='true')

# Validation croisée pour évaluer la performance du modèle
scores = cross_val_score(sgd_clf, x_train_scaled, y_train, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold :", scores)
print("Précision moyenne :", scores.mean())
print("Matrice de confusion :\n", matrix)
print(classification_report(y_val, y_pred))

#  SMOTE
sm = SMOTE(random_state=42)
x_train_res, y_train_res = sm.fit_resample(x_train_scaled, y_train)

# Entraînement du modèle sur les données résamplées
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train_res, y_train_res)

# Prédiction sur l'ensemble de test
y_pred = sgd_clf.predict(x_test_scaled)

# Validation croisée pour évaluer la performance du modèle sur les données résamplées
scores = cross_val_score(sgd_clf, x_train_res, y_train_res, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold :", scores)
print("Précision moyenne :", scores.mean())
print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred,normalize='true'))
print("\nRapport de classification :\n", classification_report(y_test, y_pred))




Scores de précision pour chaque fold : [0.85365854 0.85365854 0.25      ]
Précision moyenne : 0.6524390243902439
Matrice de confusion :
 [[0. 1.]
 [0. 1.]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.83      1.00      0.91        34

    accuracy                           0.83        41
   macro avg       0.41      0.50      0.45        41
weighted avg       0.69      0.83      0.75        41

Scores de précision pour chaque fold : [0.64285714 0.5        0.5       ]
Précision moyenne : 0.5476190476190476
Matrice de confusion :
 [[1.         0.        ]
 [0.73076923 0.26923077]]

Rapport de classification :
               precision    recall  f1-score   support

           0       0.14      1.00      0.24         3
           1       1.00      0.27      0.42        26

    accuracy                           0.34        29
   macro avg       0.57      0.63      0.33        29
weighted avg       0.91    

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## Gradient Boosting Machines

In [25]:
gb_clf = GradientBoostingClassifier(random_state=42)

# Entraînement du modèle sur les données d'entraînement
gb_clf.fit(x_train_scaled, y_train)

# Prédiction sur l'ensemble de validation
y_pred_val = gb_clf.predict(x_val_scaled)

# Calcul de la matrice de confusion sur l'ensemble de validation
matrix_val = confusion_matrix(y_val, y_pred_val)

# Validation croisée pour évaluer la performance du modèle
scores_val = cross_val_score(gb_clf, x_train_scaled, y_train, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold (validation) :", scores_val)
print("Précision moyenne (validation) :", scores_val.mean())
print("Matrice de confusion (validation) :\n", matrix_val)
print(classification_report(y_val, y_pred_val))

# Résampling avec SMOTE
sm = SMOTE(random_state=42)
x_train_res, y_train_res = sm.fit_resample(x_train_scaled, y_train)

# Utilisation de GradientBoostingClassifier pour le modèle après résampling
gb_res = GradientBoostingClassifier(random_state=42)
gb_res.fit(x_train_res, y_train_res)

# Prédiction sur l'ensemble de test
y_pred_test = gb_res.predict(x_test_scaled)

# Validation croisée pour évaluer la performance du modèle sur les données résamplées
scores_res = cross_val_score(gb_res, x_train_res, y_train_res, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold :", scores_res)
print("Précision moyenne :", scores_res.mean())
print("Matrice de confusion (test) :\n", confusion_matrix(y_test, y_pred_test))
print("\nRapport de classification (test) :\n", classification_report(y_test, y_pred_test))


Scores de précision pour chaque fold (validation) : [0.85365854 0.73170732 0.875     ]
Précision moyenne (validation) : 0.8201219512195123
Matrice de confusion (validation) :
 [[ 1  6]
 [ 1 33]]
              precision    recall  f1-score   support

           0       0.50      0.14      0.22         7
           1       0.85      0.97      0.90        34

    accuracy                           0.83        41
   macro avg       0.67      0.56      0.56        41
weighted avg       0.79      0.83      0.79        41

Scores de précision pour chaque fold : [0.88571429 0.85714286 0.88571429]
Précision moyenne : 0.8761904761904762
Matrice de confusion (test) :
 [[ 1  2]
 [ 2 24]]

Rapport de classification (test) :
               precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.92      0.92      0.92        26

    accuracy                           0.86        29
   macro avg       0.63      0.63      0.63        29
weight

## Support Vector Machines

In [23]:
# Modèle SVC
svc_clf = SVC(random_state=42)

# Entraînement du modèle sur les données d'entraînement mises à l'échelle
svc_clf.fit(x_train_scaled, y_train)

# Prédiction sur l'ensemble de test
y_pred_svc = svc_clf.predict(x_test_scaled)

# Calcul de la matrice de confusion sur l'ensemble de test
matrix_svc = confusion_matrix(y_test, y_pred_svc)

# Validation croisée pour évaluer la performance du modèle
scores_svc = cross_val_score(svc_clf, x_train_scaled, y_train, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold (SVC) :", scores_svc)
print("Précision moyenne (SVC) :", scores_svc.mean())
print("Matrice de confusion (SVC) :\n", matrix_svc)
print(classification_report(y_test, y_pred_svc))

# Résampling avec SMOTE
sm = SMOTE(random_state=42)
x_train_res, y_train_res = sm.fit_resample(x_train_scaled, y_train)

sv_res = SVC(random_state=42)
sv_res.fit(x_train_res, y_train_res)
# Prédiction sur l'ensemble de test
y_pred = sv_res.predict(x_test_scaled)

# Validation croisée avec les données résamplées
scores_res = cross_val_score(sv_res, x_train_res, y_train_res, cv=3, scoring='accuracy')

# Affichage des résultats
print("Scores de précision pour chaque fold  :", scores_res)
print("Précision moyenne  :", scores_res.mean())
print("Matrice de confusion (test) :\n", confusion_matrix(y_test, y_pred_test, normalize='true'))
print("\nRapport de classification (test) :\n", classification_report(y_test, y_pred_test))

Scores de précision pour chaque fold (SVC) : [0.85365854 0.85365854 0.875     ]
Précision moyenne (SVC) : 0.8607723577235772
Matrice de confusion (SVC) :
 [[ 0  3]
 [ 0 26]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.90      1.00      0.95        26

    accuracy                           0.90        29
   macro avg       0.45      0.50      0.47        29
weighted avg       0.80      0.90      0.85        29

Scores de précision pour chaque fold  : [0.6        0.58571429 0.57142857]
Précision moyenne  : 0.5857142857142857
Matrice de confusion (test) :
 [[1.         0.        ]
 [0.73076923 0.26923077]]

Rapport de classification (test) :
               precision    recall  f1-score   support

           0       0.14      1.00      0.24         3
           1       1.00      0.27      0.42        26

    accuracy                           0.34        29
   macro avg       0.57      0.63      0.33    

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# III - GridSearchCV

## Stochastic Gradient Descent  

In [6]:
# Stochastic Gradient Descent
param_grid_sgd = {
    'loss': ['hinge', 'modified_huber', 'squared_hinge'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000],
}

grid_search_sgd = GridSearchCV(SGDClassifier(random_state=42), param_grid_sgd, cv=3, scoring='accuracy', verbose=2)
grid_search_sgd.fit(x_train_scaled, y_train)  # entraînement 

print("Meilleurs paramètres trouvés pour SGD :")
print(grid_search_sgd.best_params_)
print("Meilleur score de validation croisée pour SGD :")
print(grid_search_sgd.best_score_)

best_model_sgd = grid_search_sgd.best_estimator_
y_pred_sgd = best_model_sgd.predict(x_test_scaled)

print("Rapport de classification sur l'ensemble de test pour SGD :")
print(classification_report(y_test, y_pred_sgd))

# Enregistrement du meilleur modèle
with open('best_sgd_model.pkl', 'wb') as file:
    pickle.dump(best_model_sgd, file)
print("Enregistrement du modèle")


Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l2; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l2; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l2; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l1; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l1; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=l1; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=elasticnet; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=elasticnet; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=1000, penalty=elasticnet; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=2000, penalty=l2; total time=   0.0s
[CV] END alpha=0.0001, loss=hinge, max_iter=2000, penalty=l2; total time=   0.0s
[CV] END alpha=0.0001, 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## Gradient Boosting Machines


In [7]:
# Gradient Boosting Machines
param_grid_gbc = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.001],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2]
}

grid_search_gbc = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_gbc, cv=3, scoring='accuracy', verbose=2)
grid_search_gbc.fit(x_train_scaled, y_train)  # entraînement 

print("Meilleurs paramètres trouvés :")
print(grid_search_gbc.best_params_)
print("Meilleur score de validation croisée :")
print(grid_search_gbc.best_score_)

best_model_gbc = grid_search_gbc.best_estimator_
y_pred_gbc = best_model_gbc.predict(x_test_scaled)

print("Rapport de classification sur l'ensemble de test :")
print(classification_report(y_test, y_pred_gbc))

# Enregistrement du meilleur modèle
with open('best_gbm_model.pkl', 'wb') as file:
    pickle.dump(best_model_gbc, file)
print("Enregistrement du modèle")

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## Support Vector Machines


In [8]:
# Support Vector Machines
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'degree': [1, 2, 3]
}

grid_search_svm = GridSearchCV(SVC(random_state=42), param_grid_svm, cv=3, scoring='accuracy', verbose=2)
grid_search_svm.fit(x_train_scaled, y_train)  # entraînement 

print("Meilleurs paramètres trouvés:")
print(grid_search_svm.best_params_)
print("Meilleur score de validation croisée :")
print(grid_search_svm.best_score_)

best_model_svm = grid_search_svm.best_estimator_
y_pred_svm = best_model_svm.predict(x_test_scaled)

print("Rapport de classification sur l'ensemble de test :")
print(classification_report(y_test, y_pred_svm))

# Enregistrement du meilleur modèle
with open('best_svm_model.pkl', 'wb') as file:
    pickle.dump(best_model_svm, file)
print("Enregistrement du modèle ")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END .......................C=0.1, degree=1, gamma=scale; total time=   0.0s
[CV] END .......................C=0.1, degree=1, gamma=scale; total time=   0.0s
[CV] END .......................C=0.1, degree=1, gamma=scale; total time=   0.0s
[CV] END ........................C=0.1, degree=1, gamma=auto; total time=   0.0s
[CV] END ........................C=0.1, degree=1, gamma=auto; total time=   0.0s
[CV] END ........................C=0.1, degree=1, gamma=auto; total time=   0.0s
[CV] END .......................C=0.1, degree=2, gamma=scale; total time=   0.0s
[CV] END .......................C=0.1, degree=2, gamma=scale; total time=   0.0s
[CV] END .......................C=0.1, degree=2, gamma=scale; total time=   0.0s
[CV] END ........................C=0.1, degree=2, gamma=auto; total time=   0.0s
[CV] END ........................C=0.1, degree=2, gamma=auto; total time=   0.0s
[CV] END ........................C=0.1, degree=2

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

# IV - Fonctionnalité supplémentaire

In [48]:
import pandas as pd
import requests

def get_weather_data(latitude, longitude, date):
    api_key = "ce3e5b714f4a4198b9585634242006"
    # URL pour récupérer les données météorologiques 
    url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={latitude},{longitude}&dt={date}"
    response = requests.get(url)
    if response.status_code == 200:     # si la réponse est OK (200), retourner les données JSON
        print("Recupération des données json réussie, voici le lien ", url)
        return response.json()
    else:
        print("Erreur lors de l'appel à l'API")

#exemple
latitude = 48.8566
longitude = 2.3522
date = "2024-06-05"
weather_data = get_weather_data(latitude, longitude, date)
print(weather_data)



Recupération des données json réussie, voici le lien  http://api.weatherapi.com/v1/history.json?key=ce3e5b714f4a4198b9585634242006&q=48.8566,2.3522&dt=2024-06-05
{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.86, 'lon': 2.35, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1718955643, 'localtime': '2024-06-21 9:40'}, 'forecast': {'forecastday': [{'date': '2024-06-05', 'date_epoch': 1717545600, 'day': {'maxtemp_c': 19.1, 'maxtemp_f': 66.4, 'mintemp_c': 11.8, 'mintemp_f': 53.2, 'avgtemp_c': 15.6, 'avgtemp_f': 60.0, 'maxwind_mph': 8.3, 'maxwind_kph': 13.3, 'totalprecip_mm': 0.33, 'totalprecip_in': 0.01, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 73, 'daily_will_it_rain': 1, 'daily_chance_of_rain': 100, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Patchy rain possible', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'uv': 4.0}, 'astro': {'sunrise': '05:49 AM', 'sun